In [ ]:
#parameters
report_id = None
auth = None

In [50]:
import pandas as pd
import requests
import msal
from io import StringIO
from powerbiclient import Report, models
from powerbiclient.authentication import DeviceCodeLoginAuthentication

In [59]:
report = Report(group_id='ceb0d1d0-6226-4aef-b245-fac7f89bb52e', report_id=report_id,  auth=auth)

In [53]:
def loaded_callback(event_details):
    print('Report is loaded')

report.on('loaded', loaded_callback)

In [54]:
def rendered_callback(event_details):
    print('Report is rendered')

report.on('rendered', rendered_callback)

In [64]:
# Render report
report

Report()

In [65]:
#GET all pages
pages = report.get_pages()
print(pages)

[{'name': 'ad7629625350de3c2f92', 'displayName': 'Customers', 'isActive': False, 'visibility': 0, 'defaultSize': {'type': 0, 'width': 1280, 'height': 720}, 'mobileSize': None, 'defaultDisplayOption': 0, 'background': None, 'wallpaper': None}, {'name': '2d7047f129c3f2484ff2', 'displayName': 'Main Offices', 'isActive': False, 'visibility': 0, 'defaultSize': {'type': 0, 'width': 1280, 'height': 720}, 'mobileSize': None, 'defaultDisplayOption': 0, 'background': None, 'wallpaper': None}, {'name': '38741aa4f17a9ebc9995', 'displayName': 'Product_Size_Weight_Color', 'isActive': False, 'visibility': 0, 'defaultSize': {'type': 0, 'width': 1280, 'height': 720}, 'mobileSize': None, 'defaultDisplayOption': 0, 'background': None, 'wallpaper': None}, {'name': '50c1abe0586dfc224c03', 'displayName': 'Sales_quantity_amount_monthCategory', 'isActive': False, 'visibility': 0, 'defaultSize': {'type': 0, 'width': 1280, 'height': 720}, 'mobileSize': None, 'defaultDisplayOption': 0, 'background': None, 'wallp

In [29]:
# Define the types of visuals to export data for
visual_types_to_export = ['barChart', 'lineChart', 'pieChart', 'clusteredColumnChart', 'table', 'tableEx', 'map', 'slicer', 'lineClusteredColumnComboChart', 'shapeMap', 'decompositionTreeVisual', 'card']

# Iterate over the visuals and export data for the specified types
for page in pages:
    page_name = page['name']
    report.set_active_page(page_name)
    page_display_name = page['displayName']
    visuals = report.visuals_on_page(page_name)
    for visual in visuals:
        if visual['type'] in visual_types_to_export:
            try:
                summarized_exported_data = report.export_visual_data(page_name, visual['name'], rows=20)
                data = StringIO(summarized_exported_data)
                # Load data into pandas DataFrame
                df = pd.read_csv(data, sep=",")
                # Store the DataFrame as a CSV file in the 'csv' folder
                csv_file_path = f'csv/{visual["type"]}_{visual["title"].replace(" ", "_")}_{visual['name']}_{page_display_name.replace(" ", "_")}.csv'
                df.to_csv(csv_file_path, index=False)
                print(f"Data for visual '{visual["title"].replace(" ", "")}' of type '{visual['type']}' exported successfully.")
            except Exception as e:
                print(f"Could not export data for visual '{visual["title"].replace(" ", "")}' of type '{visual['type']}': {e}")
    page['isActive'] = False

Data for visual 'NumberofEnterprises#,NumberofFemaleCustomers#andNumberofCustomers#' of type 'clusteredColumnChart' exported successfully.
Data for visual 'Table' of type 'tableEx' exported successfully.
Data for visual 'CountofMainOfficeStateProvincebyMainOfficeStateProvinceandMainOfficeCountry' of type 'map' exported successfully.
Data for visual 'Quantity#byWeight' of type 'pieChart' exported successfully.
Data for visual 'Quantity#bySize' of type 'pieChart' exported successfully.
Data for visual 'Quantity#byColor' of type 'pieChart' exported successfully.
Data for visual 'Table' of type 'tableEx' exported successfully.
Data for visual 'Quantity#byDSC_MONTHandCategoryName' of type 'clusteredColumnChart' exported successfully.
Data for visual 'InvoiceSales€byDSC_MONTHandCategoryName' of type 'clusteredColumnChart' exported successfully.
Data for visual 'Table' of type 'tableEx' exported successfully.
Data for visual 'Quantity#byMainOfficeCountryandMainOfficeCountry' of type 'clustere